In [ ]:
import pandas as pd

pd.options.display.max_rows = 5
pd.options.display.max_columns = 100
pd.options.display.max_colwidth = 20

In [ ]:
horse_vs_df = pd.read_csv('horse_vs_data.csv')
horse_vs_df = horse_vs_df.iloc[:,:29]
df = pd.read_pickle('df_add_id.pickle')
print(len(horse_vs_df))
print(len(df))

In [ ]:
horse_vs_df

In [ ]:
df

## メインデータの前処理

In [ ]:
def preprocessing(df):
  # 性齢
  df["性"] = df["性齢"].map(lambda x:str(x)[0])
  df["齢"] = df["性齢"].map(lambda x:str(x)[1:]).astype(int)
  # 人気
  df.loc[df['人気'] == "-", '人気'] = 99
  df["人気"] = df["人気"].map(lambda x:str(x).replace('.0', ''))
  df["人気"] = df["人気"].astype(int)
  # タイム
  df.loc[df['タイム'] == "-", 'タイム'] = "9:99.9"
  # 着差
  df.loc[df['着差'] == "-", '着差'] = "除外"
  df["着差"] = df["着差"].fillna("１着")
  # オッズ
  df.loc[df['単勝オッズ'] == "-", '単勝オッズ'] = "0.0"
  df["単勝オッズ"] = df["単勝オッズ"].astype(float)
  # 後3F
  df.loc[df['後3F'] == "-", '後3F'] = "9.9"
  df["後3F"] = df["後3F"].astype(float)
  # コーナー通過順
  df.loc[df['コーナー通過順'] == "-", 'コーナー通過順'] = "0-0-0-0"
  # 馬体重
  df = df[~(df["馬体重(増減)"].astype(str).str.contains("\."))]
  df["体重"] = df["馬体重(増減)"].str.split('(',expand=True)[0].astype(int)
  df["体重変化"] = df["馬体重(増減)"].str.split('(',expand=True)[1].str[:-1].astype(int)
  df.drop(["馬体重(増減)",'性齢'],axis=1, inplace=True)
  # 日付
  df["date"] = pd.to_datetime(df["date"],format="%Y年%m月%d日")

  return df  

## HorseResultsクラス

In [ ]:
class HorseResults:
  def __init__(self,horse_vs_df):
    self.horse_results = horse_vs_df[["日付","着順","賞金"]]
    
  def preprocessing(self):
    df = self.horse_results.copy()
    # 着順
    df["着順"] = df["着順"].map(lambda x:str(x).replace('.0', ''))
    # 日付
    df["date"] = pd.to_datetime(df["日付"])
    df.drop(["日付"],axis=1, inplace=True)
    
    self.horse_results = df


In [ ]:
hr = HorseResults(horse_vs_df)
hr.preprocessing()
hr.horse_results
hr.horse_results.index = hr.horse_results.index.map(lambda x:str(x))


In [ ]:
results_p = preprocessing(df)
len(results_p)

In [ ]:
sample_data = results_p["date"][0]
sample_data

sample_dataの日付に走った馬のリスト


In [ ]:
pd.options.display.max_rows = None

horse_id_list = results_p[results_p["date"] == sample_data]["horse_id"]
horse_id_list

In [ ]:
hr.horse_results.loc[horse_id_list]

In [ ]:
hr.horse_results

In [ ]:
horse_vs_df["2018104222"]